In [ ]:
from lxml import etree

In [ ]:
import sys
sys.path.insert(0, "../../..")

In [ ]:
from db_api.schemas import DocumentCreate

In [ ]:
d = DocumentCreate

In [ ]:
tree = etree.parse("20201121_rss_full_history")

In [ ]:
root = tree.getroot()

In [ ]:
for i, item in enumerate(root.iter("item")):
    # print(i)
    tag_to_text = {}
    for child in item:
        print("%s - %s" % (child.tag, child.text))
        tag_to_text[child.tag] = child.text
    _id = "PythonBytes:" + tag_to_text["{http://www.itunes.com/dtds/podcast-1.0.dtd}episode"]
    title = tag_to_text["{http://www.itunes.com/dtds/podcast-1.0.dtd}title"]
    author = tag_to_text["{http://www.itunes.com/dtds/podcast-1.0.dtd}author"]
    episode_number = tag_to_text["{http://www.itunes.com/dtds/podcast-1.0.dtd}episode"]
    duration_in_seconds = tag_to_text["{http://www.itunes.com/dtds/podcast-1.0.dtd}duration"] # TODO
    keywords = tag_to_text["{http://www.itunes.com/dtds/podcast-1.0.dtd}keywords"]
    text = tag_to_text["description"]
    url = tag_to_text["link"]
    break

In [ ]:
etree.tostring(item)